# A moving domain problem:
In this example we consider a moving domain problem with homogeneous Neumann boundary conditions:

\begin{alignat*}{2}
\begin{aligned}
\partial_{t} u- \Delta{u} + w \cdot \nabla{u} &= f  & in \quad \Omega(t),   \qquad  & t \in [0,T]  \\
 \nabla{u} \cdot \mathbf{n}_{\partial \Omega} &= 0  &  on \quad \partial \Omega(t),   \qquad & t \in [0,T] \\
\end{aligned}  
\end{alignat*}

The problem will be treated by a fictitious domain approach
combined with a Space-Time-DG method. We will introduce the building blocks of this method
one after another. Finally, the pieces are put together to establish the complete method.





In [1]:
import netgen.gui
%gui tk
import tkinter

from ngsolve import *
from ngsolve.comp import *
from ngsolve.utils import *
from ngsolve.internal import *
from xfem import *
from netgen.geom2d import SplineGeometry
from xfem.lsetcurv import *
from math import pi
from time import sleep

# Building blocks of the method

## Description of the geometry

We use a levelset function $\phi(t) =  \sqrt{x^2 + (y-\rho(t))^2} - r_{0}$ with
$r_{0} = 1/2$, to describe the evolving geometry. The moving domain is then given as the set of points where 
the levelset function takes negative values:
 $ \Omega(t) = \{ (x,y) \in \mathbb{R}^2 \mid \phi(x,y,t) < 0 \}$.
The time-dependent domain $\Omega(t)$ is contained in a larger, time-independent domain $\Omega = [-1,1] \times [-0.75,1.5]$.
The mesh is contructed on the background domain $\Omega$ and unfitted to $\Omega(t)$.

In [2]:
# geometry
square = SplineGeometry()
square.AddRectangle([-1,-0.75],[1,1.5],bc=1)
maxh = 0.1
mesh = Mesh (square.GenerateMesh(maxh=maxh))

# keeping track of time
t = Parameter(0)
tnew = 0
tend = 0.5
delta_t = 0.01

# the levelset function that describes the domain's evolution
r0 = 0.5
rho =  CoefficientFunction((1/(pi))*sin(2*pi*t))
levelset= CoefficientFunction(sqrt(x*x+(y-rho)*(y-rho)) -r0)

# mesh adaptation class that also calculates a P1 approximation to a levelset function
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1, discontinuous_qn=True)

# visualize the domain's evolution
while tend - tnew > delta_t/2:
    if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
        pass
    deformation = lsetmeshadap.CalcDeformation(levelset)
    Draw(IfPos(-lsetmeshadap.lset_p1,CoefficientFunction(1),CoefficientFunction(0)),mesh,"movingDom")
    tnew +=  delta_t
    t.Set(tnew)
    sleep(0.15)

bcs =  [1, 1, 1, 1]


## Finite Element Space for discretization on time slab

The problem will be treated by a Space-Time DG Method. In order to avoid 2+1 dimensional complexity, the Space-Time domain will be split into so called time slabs:
$Q^{n} = \{ (x,y,t) \mid (x,y) \in \Omega(t), \, t \in (t_{n-1},t_{n}] \} $. We will derive a suitable weak variational formulation on the $Q^{n}$ that allows to solve the problem time slab per time slab. This leads to the variational structure of a time stepping scheme. To this end, we need a Finite Element space on the time slab:

\begin{equation*}
W_{n} = \{ v: Q^{n} \rightarrow \mathbb{R} \mid v(x,t) = \sum_{m= 0}^{k_{t}}{ \phi_{m}(t) u_{m}(x)}, \, \varphi_{m} \in V_{h}, m=0, \ldots, k_{t}  \}.
\end{equation*}

The functions in $W_{n}$ are tensor products between basis functions in time $\phi_{m}(t)$ and purely space-dependent functions $u_{m}(x)$ from a standard Finite-Element Space $V_{h}$ of order $k_{s}$ defined on the background domain $\Omega$. The polynomial degrees in space and time can be prescribed independently.

In [3]:
# basis functions in time and their derivatives:
# The type of the used basis is described by the dictionaries` key. 
# Behind each such key there is a tuple. The first component of this tuple 
# gives the basis functions itself while the second yields their derivatives.
basis_in_time = { "nodal": ([lambda t: 1.0-t, lambda t: t],[lambda t: -1,lambda t: 1]) }

# choice of basis                  
phi,d_phi_ref = basis_in_time["nodal"]

k_s = 2
k_t = 1
V=H1(mesh, order = k_s, dirichlet=[])
W = FESpace([V for l in range(k_t+1)])

### Example: Interpolation in Space-Time exploiting a nodal basis 

Above we have chosen a nodal basis in time. The functions in $W_{n}$ will be of the form 
$u_{h}(x,t) = (1-t)u_{0}(x) + t u_{1}(x)$, where $u_{0},u_{1}$ come from a quadratic ($k_{s} = 2 $) Finite-Element Space on the background domain. It is convenient to work with basis functions in time that are defined on the reference time interval $[0,1]$.

To get a feeling for these type of functions let us for a moment consider the case $\Omega(t) = \Omega$. The time slabs then take the form $Q^{n} = \Omega \times (t_{n-1},t_{n}] $. Given some time dependent function $u(x,y,t) = \sin(\pi t) \sin(\pi x)^2 \sin(\pi y)^2$ we would now like to find an approximation to this function $u_{h} \in W_{n}$ in the Finite Element Space on the time slab. Making use of the nodal property leads to:
$u(\cdot,t_{n-1}) \approx u_{h}(\cdot,t = 0) = u_{0}(\cdot)$ and $u(\cdot,t_{n}) \approx u_{h}(\cdot,t = 1) = u_{1}(\cdot)$. In the following block of code $u_{h}$ is constructed and visualized.

In [4]:
# geometry
square = SplineGeometry()
square.AddRectangle([0,0],[1,1],bc=1)
maxh = 0.1
mesh = Mesh (square.GenerateMesh(maxh=maxh, quad_dominated=False))

# constants
t1  = 0
t2 = 0.5

# the time
t = Parameter(0)

# nodal basis in time               
phi = [lambda t: 1.0-t, lambda t: t]

# function to approximate
u = CoefficientFunction(sin(pi*t)*sin(pi*x)*sin(pi*x)*sin(pi*y)*sin(pi*y))

# order of FE-Spaces
k_s = 2 #space
k_t = 1 # time

# FE-Spaces
V=H1(mesh, order = k_s, dirichlet=[1])
u_eval = GridFunction(V)
Draw(u_eval,mesh,"uh")
W = FESpace([V for l in range(k_t+1)])
u_h = GridFunction(W)

# Interpolation
for i,time in enumerate([t1,t2]):
    t.Set(time)
    u_h.components[i].Set(u)

visoptions.autoscale = False
visoptions.mminval=0
visoptions.mmaxval=1.0

# Plotting u_h at different times 
for time in [i*0.1 for i in range(11)]:
    if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
        pass
    u_eval.vec[:] = 0
    for l in range(k_t+1):
        u_eval.vec.data += phi[l](time)*u_h.components[l].vec
    Redraw()
    print("t = {0}".format(t1 + time*(t2-t1)))
    sleep(0.15)

bcs =  [1, 1, 1, 1]
t = 0.0
t = 0.05
t = 0.1
t = 0.15000000000000002
t = 0.2
t = 0.25
t = 0.30000000000000004
t = 0.35000000000000003
t = 0.4
t = 0.45
t = 0.5


## Variational formulation

Now we would like to derive a suitable variational formulation on the time slabs $Q^{n}$. 

We start by multiplying the equation 
\begin{equation*}
\partial_{t} u- \Delta{u} + w \cdot \nabla{u} = f \quad  in \quad \Omega(t),   \qquad  t \in [0,T] 
\end{equation*}

by a test function $v$ and perform integration by parts
on the term that involves the Laplacian. Due to homogeneous Neumann boundary conditions this leads to:

\begin{equation*}
(\partial_{t} u, v)_{Q^n} + (\nabla{u},\nabla{v})_{Q^n}   + (w \cdot \nabla{u},v)_{Q^n} = (f,v)_{Q^n}.
\end{equation*}

At the moment this equation does not include any information from the previous time slab $Q^{n-1}$. Depending on 
the way the time derivative is interpreted, there are different ways to repair this. 

* The time derivative $\partial_{t}$ acts as a convective term in the time direction. In this sense, $\Omega(t_{n-1}) \times \{t_{n-1} \}$ is the inflow boundary of $Q^{n}$ where inflow information has to be provided. Here one adds upwind stabilization to impose weak continuity in time: $([[u]]^{n-1},v_{+}^{n-1})_{\Omega(t_{n-1})}$. The terms $v_{\pm}$ are given as the limits in time from above respectively below: $v_{\pm}^{n-1} = \lim_{s \rightarrow 0}{v(\cdot,t_{n-1} \pm s)}$. The bracket denotes the jump over the time boundary: $[[u]]^{n-1} = u_{+}^{n-1} - u_{-}^{n-1}$. The term $u_{-}^{n-1}$ is known from the previous time step and can be shifted to the right hand side.
* The operator $(\nabla,\partial_{t})$ acts as a convective term in the space-time domain. We can integrate the whole term by parts and obtain: \begin{alignat*}{2}\begin{aligned} & (\partial_{t}u,  v)_{Q^{n}} + ( w \cdot \nabla{u}, v)_{Q^{n}} \\
 &= -(u, \partial_{t} v)_{Q^{n}} + (u_{-}^{n},v_{-}^{n})_{\Omega(t_{n})} - (u_{-}^{n-1},v_{+}^{n-1})_{\Omega(t_{n-1})} - (u, \nabla{v} \cdot w)_{Q^{n}}.           \\  \end{aligned}  \end{alignat*} Here it was used that the velocity of the boundary $\partial \Omega(t)$ in normal direction coincides with $w \cdot n$ where $n$ is the normal of $\Omega(t)$.

We decide to take the second approach because it provides a conservative method. 

To approximate the arising integrals we will first apply quadrature in time.
This approach requires additional stabilization for extending the discrete solutions. Ghost-Penalty stabilization, which is known from previous examples, is appropriate for this task. Adding the stabilization, the variational formulation on the time slabs becomes:

\begin{alignat*}{3}
\begin{aligned}
 &-(u, \partial_{t} v)_{Q^{n}} + (\nabla{u},\nabla{v})_{Q^{n}} - (u, \nabla{v} \cdot w)_{Q^{n}}  + (u^{n}_{-},v^{n}_{-})_{\Omega^{n}} \\
 &+ \sum\limits_{j=1}^{k_{s}}{  \sum\limits_{F \in F_{h}}{ \gamma_{j} \int\limits_{t_{n-1}}^{t_{n}}{   \int\limits_{F}{  h_{F}^{2j-1} [[ \partial_{\mathbf{n}}^{j}u]] \, [[ \partial_{\mathbf{n}}^{j}v]]         \, d\mathbf{s} \, dt.  } }		}   }              \\
 &= (f,v)_{Q^{n}}  +  (u^{n-1}_{-},v^{n-1}_{+})_{\Omega^{n-1}}          \\
\end{aligned}  
\end{alignat*}

The facets $F_{h}$ on which the Ghost-Penalty stabilization is applied will be discussed below.


## Quadrature in time

The variational formulation involves integrals over the time slabs $Q^n$. These could be evaluated using quadrature in space-time. However, this requires Space-Time Finite Element Spaces (constructed behind the Python interface) and appropriate quadrature rules on elements that are cut in the space-time domain. The implementation is more involved and currently in development. 

Here we follow a different approach. Let us first approximate the integrals by applying quadrature only in time:
\begin{alignat*}{2}
\begin{aligned}
 (f,v)_{Q^{n}}  &= \int\limits_{t_{n-1}}^{t_{n}}{   \int\limits_{\Omega(t)}{ f(t) v(t)  \, dx \, dt.  } }  \\
   & \approx  \Delta{t} \sum\limits_{l = 0}^{ L }{ \omega_{l}   \int\limits_{\Omega(t_{n-1} + t^{l} \cdot \Delta{t})}{ f(t_{n-1} + t^{l} \cdot \Delta{t} ) v(t^{l})  \, dx.  } } 
\end{aligned}  
\end{alignat*}
The $t^{l} \in [0,1]$ are the quadrature points of a quadrature rule on the reference time interval with corresponding weights $\omega_{l}$. The time step is $\Delta{t} = t_{n} - t_{n-1}$. The remaining integrals are time-independent and can be treated in the usual manner. Higher order can be archieved by applying a mesh-deformation at the fixed time points.

The integrals above require quadrature points $t^{l}$ on the reference interval $[0,1]$. Often quadrature rules are not defined on $[0,1]$ but on some other interval, e.g. $[-1,1]$ for the Gauss-Radau quadrature. Hence, one first needs to transform the integrals to this interval according to the formula
\begin{alignat*}{2}
\begin{aligned}
 \int\limits_{a}^{b}{ g(t) \, dt  }   &= \frac{b-a}{2} \int\limits_{-1}^{1}{   g(\frac{b-a}{2}t + \frac{a+b}{2})   \, dt.  }   \\
   & \approx  \sum\limits_{l = 0}^{ L }{ \frac{b-a}{2} \omega_{l} \, g(\frac{b-a}{2}t^{l} + \frac{a+b}{2}). } 
\end{aligned}  
\end{alignat*}
In the block of code below, $\frac{b-a}{2}t^{l} + \frac{a+b}{2}$ are called the shifted quadrature points and $\frac{b-a}{2} \omega_{l}$ the shifted quadrature weights. In practice we will apply this formula with $a = 0$ and $b=1$. 
The Code below gives an example of this procedure for the case of the Gauss-Radau quadrature.


In [5]:
class quad_rule:

    def __init__(self,name,npoints):
        '''Constructor of quadrature rule. 
           This class is used for approximating the
           time integrals.
        
        Parameters
        ----------
        name : str
            Name of the quadrature rule.
        npoints : int
            Number of quadrature points.
        '''
        self.name = name
        self.npoints = npoints

        # available quadrature rules
        gauss_radau = {
            3: ([-1, (1-sqrt(6))/5, (1+sqrt(6))/5],
                [2/9, (16+sqrt(6))/18,(16-sqrt(6))/18]),
            4: ([-1, -0.575319, 0.181066, 0.822824],
                [0.125,0.657689,0.776387,0.440924])}
                
       # you may add your favourite quadrature rule here 
                
        if name == "Gauss-Radau":
            self.points = gauss_radau[npoints][0]
            self.weights = gauss_radau[npoints][1]

    def shifted_pts(self,a,b):
        '''Transformation of quadrature points to interval [a,b].
        
        Parameters
        ----------
        a : float
            Left end of the interval.
        b : float
            Right end of the interval.

        Returns
        -------
        list
            Transformed quadrature points.       
        '''
        if self.name == "Gauss-Radau":
            return [0.5*(b-a) * pt + 0.5*(b+a)  for pt in self.points]

    def shifted_weights(self,a,b):
        '''Transformation of quadrature weights to interval [a,b].
        
        Parameters
        ----------
        a : float
            Left end of the interval.
        b : float
            Right end of the interval.

        Returns
        -------
        list
            Transformed quadrature weights.
        '''
        if self.name == "Gauss-Radau":
            return [0.5*(b-a)*w for w in self.weights]

qr= quad_rule("Gauss-Radau",4)
print("Shifted points: {0}".format(qr.shifted_pts(0,1)))
print("Shifted weights: {0}".format(qr.shifted_weights(0,1)))

Shifted points: [0.0, 0.2123405, 0.590533, 0.911412]
Shifted weights: [0.0625, 0.3288445, 0.3881935, 0.220462]


### Element markers & Ghost penalty stabilization

In the stationary case, the elements of a mesh are classified according to the values that the levelset function takes on them. Elements with only negative levelset values are called `NEG`, while those with only positive values will be `POS`.
The cut elements are those on which the levelset function takes negative and positive values. These are called `IF`.

In the instationary case, the elements change their type as the domain moves through the mesh. We need to find a suitable generalization of the classification described above for elements belonging to a specific time slab. When the time slab is bounded by the time points `told` and `tnew = told + delta_t`, then we will calculate the integrals at certain points `told + t_i * delta_t` with `t_i` belonging to the reference interval $[0,1]$. At these points, the `CutInfo` provides information about the element types. Combining the information at all these points yields the desired generalization of an element type with respect to a time slab. For example, an element is classified as `hasneg_spacetime` when it has been `NEG` or `IF` on at least one of the points `told + t_i * delta_t`. When determining the interface elements `hasif_spacetime` one needs to be a bit careful. These elements include the ones that have been `IF` on at least one of the points `told + t_i * delta_t`. However, for large time steps `delta_t` it may also happen that an element jumps from `POS` to `NEG` or the other way around without ever having been in `IF` at the considered points in time. These elements need also to be included in `hasif_spacetime`.

The element markers are needed for various tasks. For example, the linear system on the time slab is solved with respect to the degrees of freedom that belong to the `hasneg_spacetime` elements. Also, we need them to determine the facets on which ghost-penalty stabilization will be carried out. The relevant facets are the ones that lie between the elements that are classified as `hasneg_spacetime` and `hasif_spacetime` and are obtained by: 
`ba_facets = GetFacetsWithNeighborTypes(mesh,a=hasneg_spacetime,b=hasif_spacetime,bnd_val_a=False,bnd_val_b=False)`

The Code below illustrates how the element markers are set and visualizes the elements belonging to `hasneg_spacetime` and `hasif_spacetime`.

In [6]:
# geometry
square = SplineGeometry()
square.AddRectangle([-1,-0.75],[1,1.5],bc=1)
maxh = 0.2
mesh = Mesh (square.GenerateMesh(maxh=maxh, quad_dominated=False))

# keeping track of time
t = Parameter(0)
tnew = 0
tend = 0.5
delta_t = 0.1
told = 0

# the levelset function that describes the domain's evolution
r0 = 0.5
rho =  CoefficientFunction((1/(pi))*sin(2*pi*t))
levelset= CoefficientFunction(sqrt(x*x+(y-rho)*(y-rho)) -r0)

# mesh adaptation class that also calculates a P1 approximation to a levelset function
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1, discontinuous_qn=True)
ci = CutInfo(mesh)

hasneg_spacetime = BitArray(ci.GetElementsOfType(NEG))
haspos_spacetime = BitArray(ci.GetElementsOfType(POS))
hasif_spacetime = BitArray(ci.GetElementsOfType(IF))


while tend - tnew > delta_t/2:
    tnew +=  delta_t
    
    hasneg_spacetime[:] = False
    haspos_spacetime[:] = False
    hasif_spacetime[:]= False
    
    # Loop over time points ti.
    # Elements are classified as hasneg_spacetime if they have 
    # been part of the active mesh on at least one of the time 
    # points ti.
    for t_i in qr.shifted_pts(0,1):
        t.Set(told + t_i * delta_t)
        deformation = lsetmeshadap.CalcDeformation(levelset)
        ci.Update(lsetmeshadap.lset_p1)
        hasneg_spacetime |= ci.GetElementsOfType(NEG)
        hasneg_spacetime |= ci.GetElementsOfType(IF)
        haspos_spacetime |= ci.GetElementsOfType(POS)
        haspos_spacetime |= ci.GetElementsOfType(IF)
        hasif_spacetime |= ci.GetElementsOfType(IF)
    
    # elements that have been in haspos and in hasneg should be marked as hasif!
    jumpels = BitArray(hasneg_spacetime)
    jumpels &= haspos_spacetime
    hasif_spacetime |= jumpels
    
    #input("Continue")
    if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
        pass
    Draw(IfPos(-lsetmeshadap.lset_p1,CoefficientFunction(1),CoefficientFunction(0)),mesh,"movingDom")
    Draw(BitArrayCF(hasneg_spacetime),mesh,"hasneg")
    Draw(BitArrayCF(hasif_spacetime),mesh,"hasif")
    Redraw()
    
    print("told = {0}".format(told))    
    told = tnew
    sleep(1.5)

bcs =  [1, 1, 1, 1]
told = 0
told = 0.1
told = 0.2
told = 0.30000000000000004
told = 0.4


# Putting all the pieces together

The building blocks from the previous section can now be assembled to treat the moving domain problem.

## Auxiliary functions

Firstly, we define some auxiliary functions. Among these are the quadrature rule as described above and the jump of the normal derivative that is needed for the Ghost-Penalty stabilization.

In [7]:
class quad_rule:

    def __init__(self,name,npoints):
        '''Constructor of quadrature rule. 
           This class is used for approximating the
           time integrals.
        
        Parameters
        ----------
        name : str
            Name of the quadrature rule.
        npoints : int
            Number of quadrature points.
        '''
        self.name = name
        self.npoints = npoints

        # available quadrature rules
        gauss_radau = {
            3: ([-1, (1-sqrt(6))/5, (1+sqrt(6))/5],
                [2/9, (16+sqrt(6))/18,(16-sqrt(6))/18]),
            4: ([-1, -0.575319, 0.181066, 0.822824],
                [0.125,0.657689,0.776387,0.440924])}
                
       # you may add your favourite quadrature rule here 
                
        if name == "Gauss-Radau":
            self.points = gauss_radau[npoints][0]
            self.weights = gauss_radau[npoints][1]

    def shifted_pts(self,a,b):
        '''Transformation of quadrature points to interval [a,b].
        
        Parameters
        ----------
        a : float
            Left end of the interval.
        b : float
            Right end of the interval.

        Returns
        -------
        list
            Transformed quadrature points.       
        '''
        if self.name == "Gauss-Radau":
            return [0.5*(b-a) * pt + 0.5*(b+a)  for pt in self.points]

    def shifted_weights(self,a,b):
        '''Transformation of quadrature weights to interval [a,b].
        
        Parameters
        ----------
        a : float
            Left end of the interval.
        b : float
            Right end of the interval.

        Returns
        -------
        list
            Transformed quadrature weights.
        '''
        if self.name == "Gauss-Radau":
            return [0.5*(b-a)*w for w in self.weights]

def dnjump(u,order,comp):
    '''Jump of normal derivative over element interfaces.
    
    Parameters
    ----------
    u : ProxyFunction
        Test or Trialfunction.
    order : int
            Order of the normal derivative
    comp : int
           Component of ProxyFunction.
           
    Returns
    -------
    ProxyFunction
        The jump of the normal derivative of the required order.    
    '''
    if order%2==0:
        return dn(u,order,comp) - dn(u.Other(),order,comp)
    else:
        return dn(u,order,comp) + dn(u.Other(),order,comp)

def power(u,p):
    if p == 0:
        return 1
    else:
        return u * power(u,p-1)

## Geometry, Problem Data & basis in time

The right hand side is chosen so that the solution is  $u(x,y,t) = \chi(\sqrt{x^2+(y-\rho(t))^2})$ with $\chi(r) = \cos^{2}(\frac{\pi r}{2 r_{0}})$.

In [8]:
# geometry
square = SplineGeometry()
square.AddRectangle([-1,-0.75],[1,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.1))

# define quadrature rule
qr= quad_rule("Gauss-Radau",3)

# constants
r0 = 0.5
r1 = 0.5*pi/r0
tend  = 0.5

# the time
t = Parameter(0)

# for evaluation of time basis functions defined on reference interval [0,1]
tref = Parameter(0) 

# available basis functions in time and their derivatives
basis_in_time = { "nodal": ([lambda t: 1.0-t, lambda t: t],[lambda t: -1,lambda t: 1]),
                  "modal": ([lambda t: 1.0, lambda t: t],[lambda t: 0.0, lambda t: 1]) }

# choice of basis                  
phi,d_phi_ref = basis_in_time["modal"]

rho =  CoefficientFunction((1/(pi))*sin(2*pi*t))
d_rho = CoefficientFunction(2*cos(2*pi*t))
w = CoefficientFunction((0,d_rho)) # convection

# the levelset function that describes the domain's evolution
levelset= CoefficientFunction(sqrt(x*x+(y-rho)*(y-rho)) -r0)

# right hand side
coeff_f = CoefficientFunction(  -(pi/r0)*r1*( sin(r1*sqrt(x*x+(y-rho)*(y-rho)))*sin(r1*sqrt(x*x+(y-rho)*(y-rho))) - cos(r1*sqrt(x*x+(y-rho)*(y-rho)))*cos(r1*sqrt(x*x+(y-rho)*(y-rho))) )  + (pi/r0)*cos(r1*sqrt(x*x+(y-rho)*(y-rho)))*sin(r1*sqrt(x*x+(y-rho)*(y-rho)))*(1/sqrt(x*x+(y-rho)*(y-rho))) )

# exact solution for monitoring the error
exact = CoefficientFunction(cos(r1*sqrt(x*x+(y-rho)*(y-rho)))*cos(r1*sqrt(x*x+(y-rho)*(y-rho))) )

# initial condition
coef_u0 = cos(r1*sqrt(x*x+y*y))*cos(r1*sqrt(x*x+y*y))

bcs =  [1, 1, 1, 1]


## Finite Element Spaces

The Finite Element Space on the time slab is defined as above with the exception that we add dgjumps here. These are needed for the jump of the normal derivative in the Ghost-Penalty terms.

In [9]:
# order of FE-Spaces
k_s = 2 #space
k_t = 1 # time

# FE-Spaces
V=H1(mesh, order = k_s, dirichlet=[],flags = {"dgjumps":True})
u0 = GridFunction(V)
W = FESpace([V for l in range(k_t+1)],flags = {"dgjumps":True}) # added DG Jumps for Ghost-penalty terms
gfu = GridFunction(W)
Draw(u0,mesh,"unew")

# stabilization parameter for Ghost-penalty
gamma_stab = [10**-l for l in range(1,k_s+1)]

## The Slab Discretization class

The slab discretization class sets up the problem on the time slabs. We do not assemble the linear system here. This happens in the `SolveProblem` function below, which contains the actual time stepping.

Here we only construct the test functions and loop over the discrete time points on the reference interval $[0,1]$ to set up the integrators. We collect the integrators into a list of dictionaries. Each such dictionary corresponds to the integrators at one time point $t_{i}$ of the reference interval. The key for this dictionary specifies on which part of the domain the integrals have to be carried out, e.g. `hasneg_spacetime`. The corresponding BitArrays are set up when the actual time stepping is done.

The time points $t_{i}$ on the reference interval, which we loop over, consist of the quadrature points and the left $t_{i} = 0$, as well as the right $t_{i} = 0$ end of the interval. These are added in the case where the quadrature rule does not contain these points.
The points on the boudary of the time interval are needed to take the contributions on the bottom and top of the time slab into account that stem from integration by parts of the time derivative.

In [10]:
class SlabDiscretization:
    '''
    Class for discretization on the time slab.
    '''
    def Update(self, delta_t):
        '''Sets up the problem on the time slabs.
    
        Parameters
        ----------
        delta_t : float
            The time step.
        '''
        self.active_domain_key = "hasneg_spacetime"
        self.bfs = []
        self.lfs = []
        # for recording on which part of the mesh the integrators are defined
        self.marked_integrators = []

        self.points_time_ref = []
        self.weights_time_ref = []

        # add dummy time point (with weight 0) if initial time is not part of time quad:
        if not 0.0 in qr.shifted_pts(0,1):
            self.points_time_ref.append(0)
            self.weights_time_ref.append(0)
            
        self.points_time_ref.extend(qr.shifted_pts(0,1))
        self.weights_time_ref.extend(qr.shifted_weights(0,1))
    
        if not 1.0 in qr.shifted_pts(0,1):
           self.points_time_ref.append(1.0)
           self.weights_time_ref.append(0.0)
        print("points_time_ref", self.points_time_ref)
        print("weights_time_ref", self.weights_time_ref)

        # mesh adaptation
        self.lsetmeshadap = LevelSetMeshAdaptation(mesh, order=k_s, threshold=0.1, discontinuous_qn=True)
        self.levelset_domain = { "levelset" : self.lsetmeshadap.lset_p1, "domain_type" : NEG }
    
        # define Bi/Linearforms:
        u = W.TrialFunction()
        v = W.TestFunction()

        h = specialcf.mesh_size
        
        # define Trial/Testfunctions 
        # these are linear combinations of tensor products (time x space)
        u_ti =   sum( [ u[n]*phi[n](tref) for n in range(k_t +1 ) ]  )  
        v_ti =   sum( [ v[n]*phi[n](tref) for n in range(k_t +1 ) ]  )
        # time-derivative (integration by parts in time)
        d_v_ti = sum( [ v[n]*d_phi_ref[n](tref) for n in range(k_t +1 ) ]  ) 
        gradu_ti = CoefficientFunction((0,0))
        gradv_ti = CoefficientFunction((0,0))
        for n in range(k_t+1):
            gradu_ti += grad(u[n])*phi[n](tref)
            gradv_ti += grad(v[n])*phi[n](tref)
        # jump of the normal derivative (needed for Ghost-Penalty stabilization)
        u_dnjump_ti = [ sum( [ phi[n](tref)*dnjump(u[n],l+1,n) for n in range(k_t+1)  ]) for l in range(k_s) ]
        v_dnjump_ti = [ sum( [ phi[n](tref)*dnjump(v[n],l+1,n) for n in range(k_t+1)  ]) for l in range(k_s) ]
        
        # loop over discrete time points to set up the integrators       
        # the actual linear system is assembled in the SolveProblem function below. 
        for t_i, omega_i in zip(self.points_time_ref, self.weights_time_ref):
                           
            ai = BilinearForm(W,symmetric=False)
            fi = LinearForm(W)
            
            marked_integrators_at_ti = {}
            marked_integrators_at_ti["hasneg_at_ti"] = []
            marked_integrators_at_ti["gpfacets_spacetime"] = []
            
            # inner terms
            inner_term =  CoefficientFunction(0)
            inner_term_rhs = CoefficientFunction(0)
            if omega_i != 0.0:
                # diffusion term
                inner_term += delta_t*omega_i*gradu_ti*gradv_ti
                # convection
                inner_term += -delta_t*omega_i*u_ti*w*gradv_ti
                # time-derivative
                inner_term += -omega_i*u_ti*d_v_ti
                # right hand side
                inner_term_rhs += delta_t*omega_i*coeff_f*v_ti
            
            # boundary terms at the time slab's bottom/top 
            # these stem from integration by parts of the time derivative            
            if t_i == 0.0:
                inner_term_rhs += u0 * v_ti                   
            if t_i == 1.0:
                inner_term += u_ti*v_ti
                
            form_inner = SymbolicBFI(self.levelset_domain,form = inner_term)
            form_inner_rhs = SymbolicLFI(self.levelset_domain, form=inner_term_rhs)
            
            ai += form_inner
            fi += form_inner_rhs
            marked_integrators_at_ti["hasneg_at_ti"].append(form_inner)
            marked_integrators_at_ti["hasneg_at_ti"].append(form_inner_rhs)
                            
            if omega_i != 0.0:
                # Ghost-Penalty Terms
                for l in range(k_s):
                    gp_term = delta_t * omega_i * gamma_stab[l] * power(h,2*l+1) * u_dnjump_ti[l] * v_dnjump_ti[l]
                    form_new_gp = SymbolicBFI( gp_term, skeleton=True )
                    ai += form_new_gp
                    marked_integrators_at_ti["gpfacets_spacetime"].append(form_new_gp)
                    
            self.bfs.append(ai)
            self.lfs.append(fi)
            self.marked_integrators.append(marked_integrators_at_ti)
            
    def __init__(self):
        pass
            
        

## The time stepping

Now the actual problem can be solved. Firstly, we update the slab discretization with the time step and define a couple of things. For example, a dummy matrix which will later later become the system matrix for solution on the time slab. Also we define the cutinfo, which provides information how elements are cut. 

Then the time loop is carried out. In any stage of this loop we have to solve a problem on the time slab bounded by the time levels `told` and `tnew`. To this end, the information at times `told + t_i * delta_t` where `t_i` are discrete time points on the reference interval has to be collected. At these points we have to query the corresponding integrators provided by the slab discretization and  assemble the matrices/vectors. These are then added to the dummy matrix/vector mentioned in the beginning that describes the linear system on the whole time slab. 

Before the system can be assembled we have to inform the bilinear and linear forms on which type of elements, e.g. "hasneg_at_ti", they are defined. This is done by the command `form.SetDefinedOnElements(markers[key])`, which has the effect that the integrator is applied on the elements/facets specified in `markers[key]`.


The required BitArrays for this are provided by cutinfo: 

`hasneg_ti = BitArray(ci.GetElementsOfType(NEG))
hasneg_ti |= ci.GetElementsOfType(IF)`.

The elements `hasneg_ti` form the so called active mesh at the time point `told + t_i * delta_t`.

In the end, the linear system has to be solved. This is done with respect to the degrees of freedom which have been part of the active mesh in at least one of the time points  `told + t_i * delta_t`. For this approach to work, additional stabilization is required. We employ Ghost-Penalty stabilization for this purpose.
 

In [11]:
def SolveProblem(sd, delta_t ):
    '''This function carries out the time stepping.
    
        Parameters
        ----------
        sd : SlabDiscretization
             Provides the integrators on the time slabs.
        delta_t : float
            Size of the time step.
            
        Returns
        -------
        float
            The L2-Error at the final point in time. 
    '''

    sd.Update(delta_t)
    u0.Update()
    gfu.Update() # also updates W

    u0.Set(coef_u0) # set initial condition

    # keeping track of time
    tstart  = 0
    told = tstart
    tnew = told
    t.Set(tstart)
    
    # Dummy-Assemble
    a =  BilinearForm(W,symmetric=False)
    f = LinearForm(W)
    a.Assemble()
    f.Assemble()
    amat = a.mat.CreateMatrix()
    fvec = f.vec.CreateVector()

    # provides information about how the elements are cut (unfitted method)
    ci = CutInfo(mesh)

    # loop for the time stepping scheme
    while tend - tnew > delta_t/2:

        #updates
        tnew +=  delta_t

        # clear storage
        amat.AsVector()[:] = 0
        fvec[:] = 0

        # marker types:
        markers = {}
        # BitArrays for marking an element's type w.r.t. the partition
        # of the domain provided by the piecewise linear levelset function
        hasneg_spacetime = BitArray(ci.GetElementsOfType(NEG))
        hasneg_spacetime[:] = False
        haspos_spacetime = BitArray(ci.GetElementsOfType(POS))
        haspos_spacetime[:] = False
        hasif_spacetime = BitArray(ci.GetElementsOfType(IF))
        hasif_spacetime[:]= False
        
        # Loop over time points ti.
        # Elements are classified as hasneg_spacetime if they have 
        # been part of the active mesh on at least one of the time 
        # points ti. The linear system for the time slab will involve 
        # the Dofs belonging to these elements.
        for t_i in sd.points_time_ref:
            t.Set(told + t_i * delta_t)
            deformation = sd.lsetmeshadap.CalcDeformation(levelset)
            ci.Update(sd.lsetmeshadap.lset_p1)
            hasneg_spacetime |= ci.GetElementsOfType(NEG)
            hasneg_spacetime |= ci.GetElementsOfType(IF)
            haspos_spacetime |= ci.GetElementsOfType(POS)
            haspos_spacetime |= ci.GetElementsOfType(IF)
            hasif_spacetime |= ci.GetElementsOfType(IF)

        # elements that have been in haspos and in hasneg should be marked as hasif!
        jumpels = BitArray(hasneg_spacetime)
        jumpels &= haspos_spacetime
        hasif_spacetime |= jumpels

        markers["hasneg_spacetime"] = hasneg_spacetime
        markers["haspos_spacetime"] = haspos_spacetime
        markers["hasif_spacetime"] = hasif_spacetime

        # Draw(BitArrayCF(hasif_spacetime),mesh,"hasifOrjump")
        # Draw(BitArrayCF(hasneg_spacetime),mesh,"hasneg")

        # determine the facets on which Ghost-Penalty stabilization is applied
        ba_facets = GetFacetsWithNeighborTypes(mesh,a=hasneg_spacetime,b=hasif_spacetime,bnd_val_a=False,bnd_val_b=False)
        markers["gpfacets_spacetime"] = ba_facets
        
        # We are now on a fixed time slab and start approximating the integrals 
        # by first applying quadrature in time.
        for i, t_i in enumerate(sd.points_time_ref):

            tref.Set(t_i)
            t.Set(told + t_i * delta_t)
            
            # calculate the mesh deformation at the current point in time
            deformation = sd.lsetmeshadap.CalcDeformation(levelset)
            
            # collect information about the current cut-situation
            ci.Update(sd.lsetmeshadap.lset_p1)
            hasneg_ti = BitArray(ci.GetElementsOfType(NEG))
            hasneg_ti |= ci.GetElementsOfType(IF) 
            haspos_ti = BitArray(ci.GetElementsOfType(POS))
            haspos_ti |= ci.GetElementsOfType(IF) 

            markers["hasneg_at_ti"] = hasneg_ti
            markers["haspos_at_ti"] = haspos_ti
            markers["hasif_at_ti"] = ci.GetElementsOfType(IF) 


            # Collect the integrators from slab discretization and inform them
            # about the current cut-situation.
            for key in markers:
                if key in sd.marked_integrators[i] and len(sd.marked_integrators[i][key])>0:
                    for form in sd.marked_integrators[i][key]:
                        form.SetDefinedOnElements(markers[key])

            # Assemble the matrices at time point ti and add them 
            # to the total system.
            # mesh adaptation
            mesh.SetDeformation(deformation)            
            sd.bfs[i].Assemble()
            sd.lfs[i].Assemble()            
            # unset mesh adaptation
            mesh.UnsetDeformation()
            amat.AsVector().data += sd.bfs[i].mat.AsVector()
            fvec.data += sd.lfs[i].vec

        # solve linear system
        active_dofs = GetDofsOfElements(W,markers[sd.active_domain_key])
        gfu.vec.data = amat.Inverse(active_dofs,"umfpack") * fvec
        tref.Set(1)
        # solution at the time slab's top
        u0.vec[:] = 0.0
        for l in range(k_t+1):
            u0.vec.data += phi[l](1.0)*gfu.components[l].vec
        
        if netgen.gui.win.tk.dooneevent(tkinter._tkinter.DONT_WAIT):
            pass
        Redraw(blocking=True)

        # update
        told = tnew
 
    # measure the error
    t.Set(tend)
    # mesh adaptation
    deformation = sd.lsetmeshadap.CalcDeformation(levelset)
    mesh.SetDeformation(deformation)
    l2error = sqrt(Integrate(sd.levelset_domain,(u0-exact)*(u0-exact),mesh))    
    print("Time: {0}, Error: {1}".format(tnew,l2error))
    # unset mesh adaptation
    mesh.UnsetDeformation()
    return l2error

##  Ready to go !

Finally, everything is set up. After constructing the slab discretization, we can solve the problem.

In [12]:
# construct slab discretization
sd = SlabDiscretization()

# solve the problem with a time step of delta_t
SolveProblem(sd, delta_t = 0.01)

points_time_ref [0.0, 0.35505102572168223, 0.8449489742783178, 1.0]
weights_time_ref [0.1111111111111111, 0.5124858261884216, 0.37640306270046725, 0.0]
SymbolicBFI-Wrapper: SymbolicCutBFI called
SymbolicLFI-Wrapper: SymbolicCutLFI called
SymbolicBFI-Wrapper: original SymbolicBFI called
SymbolicBFI-Wrapper: original SymbolicBFI called
SymbolicBFI-Wrapper: SymbolicCutBFI called
SymbolicLFI-Wrapper: SymbolicCutLFI called
SymbolicBFI-Wrapper: original SymbolicBFI called
SymbolicBFI-Wrapper: original SymbolicBFI called
SymbolicBFI-Wrapper: SymbolicCutBFI called
SymbolicLFI-Wrapper: SymbolicCutLFI called
SymbolicBFI-Wrapper: original SymbolicBFI called
SymbolicBFI-Wrapper: original SymbolicBFI called
SymbolicBFI-Wrapper: SymbolicCutBFI called
SymbolicLFI-Wrapper: SymbolicCutLFI called
Integrate-Wrapper: IntegrateX called
Time: 0.5000000000000002, Error: 0.0005276667661927184


0.0005276667661927184